# Project: A* Pathfinding visualization

This project develops the implementation of the 'A* algorithm' (commonly used in pathfinding problems) and the visualization on how the algorithm progressively gets to the right solution into a dynamic grid.
Our objective is to find the shortest possible path between two points located anywhere on a grid, given certain 'barriers' imposed in it that would not allow for a path to cross it directly. The path how ever can suround it.

- Overview of the A* algorithm

First lets define nodes and edges: For our purposes and thinking in a stochastic process,we can simply say that nodes are states that we are able to visit and edges are the connections amongst nodes that allow us to move around them. 
Through this deffinition, one can easily assume that we cannot 'visit' or stay in a edge, we can only do that on nodes. 
In the case of a grid, we will consider nodes as each square inside of it and the edges will be the invisible paths amongst each square.

Is important to say that this is an 'informed search' algorithm, which means it will not test each possible path to arrive from the defined point A to the defined point B, it will only be doing the necessary trials to be improving 
its current proposed path along the way.

- Requierements for the algorithm

1. We start by defining the start node (A).
2. We also define the end node (Z).
3. We define the function that we will use to measure absolute distances in points: This function will not take into account the barriers, it will simply take into account the distance amongst the current node and the end node. 
Usually it is used the euclidean distance or manhattan distance as functions. We will call this function 'H(n)': The approximate absolute distance from node n to the end node (Z).
4. We define the following function 'G(n)': The shortest distance (available, taking into account the grid and the barriers) of the current node n to the start node (A)
5. We define the scoring function of the node as 'F(n) = H(n) + G(n)'

- Development of the algorithm

1. Generate a DataFrame (FX) which contains as rows all possible nodes for the problem and as columns: F|G|H|Last (where Last is the name of the node where the current node comes from).
2. Fill FX rows on default values as follows: columns F|G|H for the start node (A) = 0. columns F|G|H for any other node = inf
3. Position into node n = 0 (A)
Identify all possible neighbours of node n 
Position into one of the neighbours of node n = 0
How long is this node from the initial node given that one must pass on the path from the previous node ? (on the grid each edge always has a value of 1)
Is this length from node n (1) shorter than the current G score of the current node (the neighbour selected)?
<br/>
If yes: 
    3.1 replace G score of the current node (neighbour selected) for the edge length (on the grid always 1) THIS MEANS THAT THE BEST PATH WE KNOW (CURRENTLY) TO GET TO CURRENT NODE (NEIGHBOUR SELECTED) IS PATH NODE N = 0 (A) -> CURRENT NODE
    3.2 replace H score of the current node (neighbour selected) by using the H function (manhattan or euclidean) and computing the distance to end node (Z)
    3.3 Update F  = G + H
    3.4 In 'Last' column, write the name of the node where it comes from (in this case is node n = 0 (A))
If no:
Pass to the following neighbour to evaluate
<br/>
<br/>
4. Once all nodes evaluated, look for the minimum F score
5. Track back all the path that got us into that F score
6. This is the optimum path !

Note we have not yet introduced the barriers problem, which will be considered later on.

In [13]:
#You need to install pygame to do the visualization
pip install pygame

SyntaxError: invalid syntax (<ipython-input-13-51ade3fed2f4>, line 2)

In [1]:
import pygame
import math
from queue import PriorityQueue


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
WIDTH = 800
WIN = pygame.display.set_mode((WIDTH,WIDTH)) #We are setting here a 800x800 width display.
pygame.display.set_caption("A* Path Finding Algorithm")
#Introduce Color Codes will help you give a more visual aspect on the visual implementation

RED = (255,0,0) #This will be the color of rejects; this means that the square has already been evaluated and is not a good fit for the optimal solution.
GREEN = (0,255,0)


BLUE = (0,255,0)
YELLOW = (255,255,0)
WHITE = (255,255,255) #This will be the defaulted value of the node; this means that that square has not yet been evaluated.
BLACK  = (0,0,0) #This will be the color of the barrier; this means that the node cannot be evaluated.
PURPLE = (128,0,128) #This will be the color of the path; this means that the node has been evaluated and up to this point is considered as part of the optimal path.
ORANGE = (255,165,0) #This will be the start node color; only one node can have this color.
GREY = (128,128,128)
TURQUOISE = (64,224,208)

In [3]:
#We need to keep track of the characteristics of each node we will be testing each iteration of the algorithm, so it is useful to define a class of those nodes.
#It should have the following characteristics
#Color, Position, Neighbours, Width

class Spot:
    def __init__(self, row, col, width, total_rows): #This can be interpreted as a constructor function if you know C++ class deffinitions.
        self.row = row
        self.col = col
        self.x = row*width #Coordinate x in the grid, given that 'width' = WIDTH/N where N is the number of cubes you wish to have inside the grid.
        self.y = col*width #Coordinate y in the grid ""
        self.color = WHITE #Defaulted value should be white
        self.neighbours = [] #As defaule we use a blank list
        self.width = width
        self.total_rows = total_rows

#After developing the constructor, we will now build some methods for the 'Spot' objects:
    def get_pos(self): #You will be getting the coordinates of the node.
        return self.row, self.col
    def is_closed(self): #This is a boolean that will show TRUE when the current node has already been discarded
        return self.color == RED
    def is_barrier(self): #This is a boolean that will show TRUE when the current node is a barrier
        return self.color == BLACK
    def is_start(self): #This is a boolean that will show TRUE when the current node is the star node
        return self.color == ORANGE
    def is_end(self): #This is a boolean that will show TRUE when the current node is the end node
        return self.color == TURQUOISE

#We then create methods that can actually modify the current color stated of the node with the same definitions from above
    def reset(self): 
        self.color = WHITE
    def make_closed(self): 
        self.color = RED
    def make_open(self): 
        self.color = GREEN
    def make_barrier(self): 
        self.color = BLACK
    def make_start(self): 
        self.color = ORANGE
    def make_end(self): 
        self.color = TURQUOISE
    def make_path(self): 
        self.color = PURPLE
    def draw(self, win): #This method is the one that actually draws the cube with its (current) properties on the grid, where you have to make reference to the window or 'canvas' you created earlier
        pygame.draw.rect(win,self.color,(self.x,self.y,self.width,self.width)) #Here, you must write in the first argument the window where you will draw (pre coded), the color of the node, and the coordinates (x,y) as well as the width of each coordinate. Passing the same width makes it a square
    def update_neighbours(self,grid): #This function will update the available neighbours for the current node. Recall that barriers cannot be taken into account as neighbours to be evaluated. 
        self.neighbours = []
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier():#DOWN
            self.neighbours.append(grid[self.row + 1][self.col])
        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier():#UP
            self.neighbours.append(grid[self.row - 1][self.col])   
        if self.col > 0 and not grid[self.row][self.col -1].is_barrier():#LEFT
            self.neighbours.append(grid[self.row][self.col -1])
        if self.col < self.total_rows -1 and not grid[self.row][self.col +1].is_barrier():#RIGHT
            self.neighbours.append(grid[self.row][self.col +1])

In [4]:
object1 = Spot(3,4,10,100) #This exemplifies the creating of a node object.
object1.make_start()
object1.color

(255, 165, 0)

In [5]:
#Now that we have finished defining the class for the nodes, we must define the absolute distance function we previously mentioned. 
def h(p1,p2): #We will use the manhattan distance formula, but it can be modified
    x1,y1 = p1
    x2,y2 = p2
    return abs(x1-x2) + abs (y1-y2)
h((1,1),(2,5))

5

In [6]:
#Lets define the actual algorithm to find the optimal path:
def reconstruct_path(came_from, current, draw):
    while current in came_from:
        current = came_from[current]
        current.make_path()
        draw()
def algorithm(draw,grid,start,end):
    count = 0
    open_set = PriorityQueue()#Kind of object which treats specially lists (queues)
    open_set.put((0,count,start))
    came_from = {} #Keeps track of which node we come from before
    g_score = {spot: float("inf") for row in grid for spot in row} #Default inf values. Shortest distance to get from the start node to the currentnode
    g_score[start] = 0
    f_score = {spot: float("inf") for row in grid for spot in row} #Approximate closest distance from the current node to the end node.
    f_score[start] = h(start.get_pos(), end.get_pos())
    
    open_set_hash = {start}
    
    while not open_set.empty(): #Run until there all possible spots are exhausted
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        current = open_set.get()[2] #Saves the current node we are looking at. If two nodes have the same score, we take the first one that we found
        open_set_hash.remove(current)
        if current == end: #If the node we found is actually the end node, then we have just found the best path
            reconstruct_path(came_from, end, draw)
            end.make_end()
            return True
        for neighbour in current.neighbours:
            temp_g_score = g_score[current] + 1
            if temp_g_score < g_score[neighbour]: #If we found a better node, then update the value
                came_from[neighbour] = current
                g_score[neighbour] = temp_g_score
                f_score[neighbour] = temp_g_score  + h(neighbour.get_pos(), end.get_pos())
                if neighbour not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbour],count,neighbour))
                    open_set_hash.add(neighbour)
                    neighbour.make_open()
        draw()
        if current != start:
            current.make_closed()
    return  False

In [7]:
#We have the visualization of the grid where we will be able to show the development of the algorithm in finding the optimal path, however, we need not only a visualization but an actual ordered object that can manipulate and store the status and coordinates of each node. So we will create a function that generates such grid in a list:
def make_grid(rows,width): #recall that the grid is squared to the number of rows must be equal to the number of columns. The width parameter is the width of the ENTIRE grid. 
    grid = []
    gap = width // rows #This will show us the size it must have each node in an integer value
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Spot(i,j,gap,rows) # So here we are filling each space of the list with spot objects ! 
            grid[i].append(spot)
    return grid
print(make_grid(3,1))


[[<__main__.Spot object at 0x000001577C96EF08>, <__main__.Spot object at 0x000001577C96E208>, <__main__.Spot object at 0x000001577C96E2C8>], [<__main__.Spot object at 0x000001577C96EA08>, <__main__.Spot object at 0x000001577C96E248>, <__main__.Spot object at 0x000001577C96EB48>], [<__main__.Spot object at 0x000001577C96EDC8>, <__main__.Spot object at 0x000001577C96E1C8>, <__main__.Spot object at 0x000001577C96E688>]]


In [8]:
#We need to include in the visualization the grey lines in the grid to distinguish each node from each other. Otherwise, everything will be as a blank canvas.
def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GREY, (0,i * gap), (width, i * gap)) #Drawing the horizontal lines of each row
    for j in range(rows):
        pygame.draw.line(win, GREY, (j * gap, 0), (j * gap, width)) #Drawing the vertical lines of each column

In [9]:
#Now, we build a function that takes the previous drawing functions into action:
def draw(win, grid, rows, width):
    win.fill(WHITE) #First, you put everything in white
    for row in grid: #The grid must be previously built
        for spot in row: #The spot class bust be already run
            spot.draw(win)
    draw_grid(win,rows,width) #Then you draw the grey lines in the grid
    pygame.display.update() #Erase any changes from before from the canvas and update it with the recent changes we just did

#Now, we need to generate a function that allows us to automatically change the color to black of a current node if it gets clicked 
#This will allow us to dinamically generate barriers. 

def get_clicked_pos(pos,rows, width):
    gap = width // rows #Recall this is the size of the node
    y, x = pos 
    row = y // gap #Shows us where you are clicking, taking into account the width of each node
    col = x // gap #Shows us where you are clicking, taking into account the width of each node
    return row, col 


In [10]:
#We now create the main function, which will help us run everything
def main(win, width):
    ROWS = 50 #Number of rows in the grid. You can change this at any time
    grid = make_grid(ROWS, width) #Array of spots
    
    start = None
    end = None
    
    run = True
    started = False
    while run:
        draw(win, grid, ROWS, width) #Draw the visualization of the grid
        for event in pygame.event.get(): #An event is anything that the user did through the computer once the program is initialized, like for example a click somewhere, pushing any key of the keyboard, etc
            if event.type == pygame.QUIT:
                run = False #Finish the while conditions 

            if pygame.mouse.get_pressed()[0]: #If the LEFT [0] click of the mouse got pressed
                pos = pygame.mouse.get_pos() #Gets the coordinates of where the user clicked 
                row, col = get_clicked_pos(pos,ROWS,width) #Transforms such coordinates into row and column numbers of where the user clicked inside the grid
                spot = grid[row][col]
                if (not start) and spot != end: #If the algorithm has not yet started, then make this initial click the start node
                    start = spot
                    start.make_start()
                elif (not end) & ( spot != start): #If there already is a deffinition of the start, then make this click the end node
                    end = spot
                    end.make_end()
                elif  spot != end and spot != start:#If there already is a start and end node, then this should be a barrier
                    spot.make_barrier() 
            elif pygame.mouse.get_pressed()[2]: #IF the RIGHT [2] click of the mouse got pressed
                pos = pygame.mouse.get_pos() #Gets the coordinates of where the user clicked 
                row, col = get_clicked_pos(pos,ROWS,width) #Transforms such coordinates into row and column numbers of where the user clicked inside the grid
                spot = grid[row][col]
                spot.reset()#We will use this right click to reset the spot into white even for the start and end spots.
                if spot == start:
                    start = None
                elif spot == end:
                    end = None
#Visualization is all done ! Now, lets start writing the algorithm to find the optimal path:
            if event.type == pygame.KEYDOWN: #We will evaluate the signals from pressing the keyboard:
                if event.key == pygame.K_SPACE and start and end: #If you press the space key and the program has not yet started, we will start the algorithm
                    for row in grid:
                        for spot in row: 
                            spot.update_neighbours(grid) #UPDATE ALL NEIGHBOURS OF EACH SPOT IN THE GRID, TAKING CARE OF THE BARRIERS AND LIMITS OF THE GRID.
                    algorithm(lambda: draw(win, grid, ROWS, width), grid, start, end)
                if event.key == pygame.K_c:
                    start = None
                    end = None
                    grid = make_grid(ROWS, width)
    pygame.quit() #Finish

main(WIN, WIDTH)
                

KeyboardInterrupt: 